# 2019-12-20-coffea-demo

This demo of the new Awkward Array was presented on December 20, 2019, before the final 1.0 version was released. Some interfaces may have changed. To run this notebook, make sure you have version 0.1.36 ([GitHub](https://github.com/scikit-hep/awkward-1.0/releases/tag/0.1.36), [pip](https://pypi.org/project/awkward1/0.1.36/)) by installing

```bash
pip install 'awkward1==0.1.36'
```

The basic concepts of Awkward arrays are presented on the [old Awkward README](https://github.com/scikit-hep/awkward-array/tree/0.12.17#readme) and the motivation for a 1.0 rewrite are presented on the [new Awkward README](https://github.com/scikit-hep/awkward-1.0/tree/0.1.32#readme).

In [1]:
# The base of the GitHub repo is two levels up from this notebook.
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))

## High-level array class

The biggest user-facing change is that, instead of mixing NumPy arrays and `JaggedArray` objects, the new Awkward has a single `Array` class.

In [2]:
import numpy as np
import awkward1 as ak

array1 = ak.Array([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
array1

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5]] type='3 * var * float64'>

In [3]:
array2 = ak.Array([{"x": 0, "y": []}, {"x": 1, "y": [1.1]}, {"x": 2, "y": [1.1, 2.2]}])
array2

<Array [{x: 0, y: []}, ... y: [1.1, 2.2]}] type='3 * {"x": int64, "y": var * flo...'>

The same `Array` class is used for all data structures, such as the array of lists in `array1` and the array of records in `array2`.

There won't be any user-level functions that apply to some data types and not others. The result of an operation is likely type-dependent, but its accessibility is not. (At this time, the only existing operations are conversions and descriptions.)

(Incidentally, the width of that string representation is exactly large enough to fit into GitHub and StackOverflow text boxes without scrolling.)

In [4]:
ak.tolist(array1)

[[1.1, 2.2, 3.3], [], [4.4, 5.5]]

In [5]:
ak.tojson(array1)

'[[1.1,2.2,3.3],[],[4.4,5.5]]'

In [6]:
ak.tolist(array2)

[{'x': 0, 'y': []}, {'x': 1, 'y': [1.1]}, {'x': 2, 'y': [1.1, 2.2]}]

In [7]:
ak.tojson(array2)

'[{"x":0,"y":[]},{"x":1,"y":[1.1]},{"x":2,"y":[1.1,2.2]}]'

In [8]:
ak.typeof(array1)

3 * var * float64

In [9]:
ak.typeof(array2)

3 * {"x": int64, "y": var * float64}

(Data types are described using the [datashape language](https://datashape.readthedocs.io/en/latest/). Some Awkward features are [not expressible](https://github.com/blaze/datashape/issues/237) in the current datashape specification, so they're expressed in an extension of the language using the same style of syntax.)

The next major change in interface is that operations on arrays, such as `ak.tolist` and `ak.typeof` above, are free-standing functions, rather than class methods. This is because it's desirable to put domain specific (e.g. physics) methods on the array object itself; using free-standing functions for array manipulations avoids conflicts. For example,

   * `ak.cross(array1, array2)` is an array-manipulation function (the cross-join of `array1` and `array2`)
   * `array1.cross(array2)` could be a user-defined method, such as the 3D cross-product, if `array1` and `array2` represent (arrays of) 3D vectors.
   * `array1.somefield` is a shortcut for `array1["somefield"]`.

## Low-level array classes

The old `JaggedArray` and `Table` are still available, but you have to ask for them explicitly with `layout`. They're not "private" or "internal implementations" (there's no underscore in `layout`): they're public for frameworks like Coffea but hidden from data analysts.

As such, their string representations have more low-level detail: the contents of indexes, rather than what they mean as high-level types. (The XML formatting is just an elaboration on Python's angle-bracket convention for `repr` and the fact that we need to denote nesting.)

In [10]:
array1.layout

<ListOffsetArray64>
    <type>var * float64</type>
    <offsets><Index64 i="[0 3 3 5]" offset="0" at="0x5648829fb770"/></offsets>
    <content><NumpyArray format="d" shape="5" data="1.1 2.2 3.3 4.4 5.5" at="0x564882a02340">
        <type>float64</type>
    </NumpyArray></content>
</ListOffsetArray64>

In [11]:
array2.layout

<RecordArray>
    <type>{"x": int64, "y": var * float64}</type>
    <field index="0" key="x">
        <NumpyArray format="l" shape="3" data="0 1 2" at="0x564882a06780">
            <type>int64</type>
        </NumpyArray>
    </field>
    <field index="1" key="y">
        <ListOffsetArray64>
            <type>var * float64</type>
            <offsets><Index64 i="[0 0 1 3]" offset="0" at="0x564882a0d240"/></offsets>
            <content><NumpyArray format="d" shape="3" data="1.1 1.1 2.2" at="0x564882a0f250">
                <type>float64</type>
            </NumpyArray></content>
        </ListOffsetArray64>
    </field>
</RecordArray>

These classes are defined in C++ and wrapped by pybind11. The `awkward1.Array` class is pure Python. Many of the same operations work for layout classes, though less attention has been paid to its interface.

In [12]:
ak.typeof(array1)

3 * var * float64

In [13]:
ak.typeof(array1.layout)

var * float64

In [14]:
ak.tojson(array1)

'[[1.1,2.2,3.3],[],[4.4,5.5]]'

In [15]:
ak.tojson(array1.layout)

'[[1.1,2.2,3.3],[],[4.4,5.5]]'

In [16]:
array1.layout.tojson()

'[[1.1,2.2,3.3],[],[4.4,5.5]]'

## Behavioral mix-ins

The primary use of Awkward arrays so far has been to represent arrays or jagged arrays of physics objects with physics methods on the array objects themselves. In Awkward 0.x, this was implemented with Python multiple inheritance, but that's a Python-only solution that can't be passed into C++ (and it was brittle: easy for an array component to lose its methods).

Now behavioral mix-ins are a "first class citizen," built into Awkward 1.0's type system.

In [17]:
class PointClass(ak.Record):
    def __repr__(self):
        return "<Point({}, {})>".format(self["x"], self["y"])
    
    def mag(self):
        return abs(np.sqrt(self["x"]**2 + self["y"]**2))

ak.namespace["Point"] = PointClass

In [18]:
array3 = ak.Array([{"x": 1, "y": 1.1}, {"x": 2, "y": 2.2}, {"x": 3, "y": 3.3}])
array3

<Array [{x: 1, y: 1.1}, ... {x: 3, y: 3.3}] type='3 * {"x": int64, "y": float64}'>

In [19]:
array3.layout.type

{"x": int64, "y": float64}

Types can have arbitrary parameters, which modify their meaning. These types are JSON-encoded and passed through C++ or wherever the arrays get sent.

In [20]:
pointtype = array3.layout.type
pointtype["__class__"] = "Point"
pointtype

struct[["x", "y"], [int64, float64], parameters={"__class__": "Point"}]

In [21]:
pointtype["__str__"] = "PointType[{}, {}]".format(pointtype.field("x"), pointtype.field("y"))
pointtype

PointType[int64, float64]

In [22]:
# There will be a better interface for assigning types...
array4 = ak.Array(array3.layout, type=ak.ArrayType(pointtype, len(array3.layout)))
array4

<Array [<Point(1, 1.1)>, ... <Point(3, 3.3)>] type='3 * PointType[int64, float64]'>

In [23]:
[x.mag() for x in array4]

[1.4866068747318506, 2.973213749463701, 4.459820624195552]

The elements of this array are `PointClass` instances because the `__class__` parameter is `"Point"`, a name that is recognized in Awkward's class namespace.

In [24]:
ak.namespace

{'char': awkward1.behavior.string.CharBehavior,
 'string': awkward1.behavior.string.StringBehavior,
 'Point': __main__.PointClass}

As you can see, arrays of characters and variable-length strings are implemented as mix-ins. Apart from this type annotation, a string is just a jagged array of 8-bit integers.

In [25]:
array5 = ak.Array(["Daisy", "Daisy", "give", "me", "your", "answer", "do."])
array5

<Array ['Daisy', 'Daisy', ... 'answer', 'do.'] type='7 * string'>

In [26]:
array5.layout

<ListOffsetArray64>
    <type>string</type>
    <offsets><Index64 i="[0 5 10 14 16 20 26 29]" offset="0" at="0x564882a1ec60"/></offsets>
    <content><NumpyArray format="B" shape="29" data="0x 44616973 79446169 73796769 76656d65 796f7572 616e7377 6572646f 2e" at="0x564882a05490">
        <type>utf8</type>
    </NumpyArray></content>
</ListOffsetArray64>

In [27]:
ak.tolist(array5.layout)

[[68, 97, 105, 115, 121],
 [68, 97, 105, 115, 121],
 [103, 105, 118, 101],
 [109, 101],
 [121, 111, 117, 114],
 [97, 110, 115, 119, 101, 114],
 [100, 111, 46]]

In [28]:
ak.tolist(array5.layout[:, 1:])

[[97, 105, 115, 121],
 [97, 105, 115, 121],
 [105, 118, 101],
 [101],
 [111, 117, 114],
 [110, 115, 119, 101, 114],
 [111, 46]]

In [29]:
array5[:, 1:]

<Array ['aisy', 'aisy', ... 'nswer', 'o.'] type='7 * string'>

The string interpretation is _only_ applied to the high-level `Array` and _not_ to the layout classes. Thus,

   * superclass-based mix-ins don't have to be captured and passed on through all operations,
   * mix-ins can pass through C++ because they are only JSON-encoded type parameters, not a Python class,
   * mix-in classes don't have to be dynamically generated (`PointClass` has a "fixed address" for pickling),
   * the mechanism for array mix-ins (e.g. `string`) is the same as for producing objects (e.g. `PointClass`); there is no need to introduce an `ObjectArray`,
   * unlike old Awkward's `ObjectArray`, these records remain Awkward data structures when instantiated.

In [30]:
array4[2]

<Point(3, 3.3)>

In [31]:
array4[2].layout

<Record at="2">
    <RecordArray>
        <type>PointType[int64, float64]</type>
        <field index="0" key="x">
            <NumpyArray format="l" shape="3" data="1 2 3" at="0x5648829f85e0">
                <type>int64</type>
            </NumpyArray>
        </field>
        <field index="1" key="y">
            <NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x564882a1a820">
                <type>float64</type>
            </NumpyArray>
        </field>
    </RecordArray>
</Record>

## Agreement with NumPy

Awkward array represents a superset of NumPy's core, so it must return the same results as NumPy. This was tricky in old Awkward, when we restricted ourselves to vectorized functions, so there are hidden limitations: slices were limited to depth `2`, concatenation was limited to `axis <= 1`, and `choose(n)` was limited to `n < 5`. But now that we can write compiled for loops, there are no such limitations.

In [32]:
deepnumpy = np.arange(2*3*5*7).reshape(2, 3, 5, 7)
deepawkward = ak.Array(deepnumpy)
deepawkward

<Array [[[[0, 1, 2, 3, ... 207, 208, 209]]]] type='2 * 3 * 5 * 7 * int64'>

In [33]:
deepnumpy[1:, :2, [4, 1, 1, -2], ::-1]

array([[[[139, 138, 137, 136, 135, 134, 133],
         [118, 117, 116, 115, 114, 113, 112],
         [118, 117, 116, 115, 114, 113, 112],
         [132, 131, 130, 129, 128, 127, 126]],

        [[174, 173, 172, 171, 170, 169, 168],
         [153, 152, 151, 150, 149, 148, 147],
         [153, 152, 151, 150, 149, 148, 147],
         [167, 166, 165, 164, 163, 162, 161]]]])

In [34]:
deepawkward[1:, :2, [4, 1, 1, -2], ::-1]

<Array [... 166, 165, 164, 163, 162, 161]]]] type='1 * 2 * 4 * 7 * int64'>

In [35]:
ak.tolist(deepnumpy[1:, :2, [4, 1, 1, -2], ::-1]) == ak.tolist(deepawkward[1:, :2, [4, 1, 1, -2], ::-1])

True

## Creating arrays

A few of the examples above create arrays by passing them to the `Array` constructor. This is like old Awkward's `fromiter` function. In fact, new Awkward has a `fromiter` function, but it's implicitly called by the `Array` constructor.

In [36]:
# Calls ak.fromiter, which converts rowwise → columnar data.
ak.Array([[1.1, 2.2, 3.3], [], [4.4, 5.5]])

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5]] type='3 * var * float64'>

In [37]:
# Calls ak.fromjson, which deserializes.
ak.Array("[[1.1, 2.2, 3.3], [], [4.4, 5.5]]")

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5]] type='3 * var * float64'>

In [38]:
# Calls ak.fromnumpy, which views.
nparray = np.array([[1.1, 2.2, 3.3], [4.4, 5.5, 6.6]])
akarray = ak.Array(nparray)
akarray

<Array [[1.1, 2.2, 3.3], [4.4, 5.5, 6.6]] type='2 * 3 * float64'>

In [39]:
nparray[0, 1] = 999
akarray

<Array [[1.1, 999, 3.3], [4.4, 5.5, 6.6]] type='2 * 3 * float64'>

You can also build these manually from the layouts, but it's a lot of work!

In [40]:
content = ak.layout.NumpyArray(np.array([1.1, 2.2, 3.3, 4.4, 5.5]))
offsets = ak.layout.Index64(np.array([0, 3, 3, 5], dtype=np.int64))   # match 64-bit to 64-bit to avoid copy
listoffsetarray = ak.layout.ListOffsetArray64(offsets, content)
listoffsetarray

<ListOffsetArray64>
    <offsets><Index64 i="[0 3 3 5]" offset="0" at="0x5648829f23f0"/></offsets>
    <content><NumpyArray format="d" shape="5" data="1.1 2.2 3.3 4.4 5.5" at="0x564882a22bf0"/></content>
</ListOffsetArray64>

In [41]:
ak.Array(listoffsetarray)

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5]] type='3 * var * float64'>

## FillableArray

The `fromiter` algorithm has been expanded into a builder interface, so that you can accumulate Awkward arrays.

In [42]:
builder = ak.FillableArray()

for i in range(10):
    builder.beginrecord()
    builder.field("x")
    builder.real(np.random.normal())
    builder.field("y")
    builder.beginlist()
    for j in range(np.random.poisson(2.5)):
        builder.integer(np.random.randint(0, 10))
    builder.endlist()
    builder.endrecord()

builder

<FillableArray [{x: -0.25, y: [8, 2, 1, ... 6, 6, 4]}] type='10 * {"x": float64,...'>

This is not a regular array, but you can `snapshot` it to get one (and keep filling the `builder`). A `snapshot` does not copy array data: if you take several snapshots while filling, they _might_ share data. (And they _might_ not, if it has allocated new buffers to grow beyond its reserved space!)

In [43]:
array6 = builder.snapshot()
array6

<Array [{x: -0.25, y: [8, 2, 1, ... 6, 6, 4]}] type='10 * {"x": float64, "y": va...'>

In [44]:
ak.tolist(array6)

[{'x': -0.2502049714838009, 'y': [8, 2, 1, 4]},
 {'x': 1.8220652176976366, 'y': [8]},
 {'x': -0.5538841098196848, 'y': [5]},
 {'x': -0.8057139895507321, 'y': []},
 {'x': 0.8116459346283147, 'y': [8]},
 {'x': 0.3021475387471296, 'y': [2, 6]},
 {'x': 0.8321850650446265, 'y': [7, 2]},
 {'x': -0.1662542511062629, 'y': [3, 9, 2, 8]},
 {'x': 0.981756807450879, 'y': [8]},
 {'x': 0.5787180852091849, 'y': [8, 1, 9, 6, 6, 4]}]

In [45]:
ak.typeof(array6)

10 * {"x": float64, "y": var * int64}

The array that you produce can have nested structure, as shown above. The structure was determined by the order in which `builder` methods were called.

You can write algorithms that build arrays as if you were printing out JSON:

   * call `beginlist()` instead of printing `"["`,
   * call `endlist()` instead of printing `"]"`,
   * call `beginrecord()` instead of printing `"{"`,
   * call `endrecord()` instead of printing `"}"`,
   * call `field(key)` instead of printing `"key":`, etc.

In [46]:
deepbuilder = ak.FillableArray()

def deepnesting(depth):
    if depth == 0:
        deepbuilder.integer(np.random.randint(0, 10))
    else:
        deepbuilder.beginlist()
        for j in range(np.random.poisson(2.5)):
            deepnesting(depth - 1)
        deepbuilder.endlist()

deepnesting(5)

In [47]:
ak.tolist(deepbuilder.snapshot())

[[[[[[4, 1], [1], [3], [4, 3], [7, 8]], [[2, 2, 7, 2, 3]]],
   [[[2, 8, 0, 5, 4], [8], [3, 2]],
    [[9, 0, 2], [0]],
    [[0, 9, 9], [6], [9], [6, 3, 3], [7, 3, 5, 0, 1, 0, 6]],
    [],
    [[8, 9, 9], [7, 2, 5, 0], [7], [9, 9, 2], [7, 1, 0], [3, 7]],
    [[5, 9, 2, 8, 0]]]],
  [[[[9, 2, 1], [6, 7, 9]]],
   [[[8, 7, 1, 2],
     [7, 7, 5, 8, 7, 6],
     [0, 7, 7],
     [6, 0, 1, 9],
     [6, 6],
     [6, 7, 7, 8, 5]],
    [[9, 4, 7, 6, 8], [6], [9, 6, 1], [4, 7, 3]],
    [[4, 1], [7, 6, 0], [2], [0], [4], [0, 7, 9]]],
   [[[5, 6, 0, 2], [4]], [[6, 9, 3, 0, 1], [2, 0, 5, 6], [5, 0, 8], [0]], []]],
  [[[[0], [3, 2]], [[6], [5, 1, 6]]],
   [[[], [8, 9]], [[6, 5], [9, 5], [3, 8, 3, 9, 0]]],
   [[[5, 3, 9], [5, 1, 6]]]]]]

In [48]:
ak.typeof(deepbuilder)

1 * var * var * var * var * var * int64

In [49]:
deepbuilder.snapshot().layout

<ListOffsetArray64>
    <type>var * var * var * var * var * int64</type>
    <offsets><Index64 i="[0 3]" offset="0" at="0x564882a45f90"/></offsets>
    <content><ListOffsetArray64>
        <type>var * var * var * var * int64</type>
        <offsets><Index64 i="[0 2 5 8]" offset="0" at="0x564882a47fa0"/></offsets>
        <content><ListOffsetArray64>
            <type>var * var * var * int64</type>
            <offsets><Index64 i="[0 2 8 9 12 15 17 19 20]" offset="0" at="0x564882a49fb0"/></offsets>
            <content><ListOffsetArray64>
                <type>var * var * int64</type>
                <offsets><Index64 i="[0 5 6 9 11 ... 49 51 53 56 58]" offset="0" at="0x564882a4bfc0"/></offsets>
                <content><ListOffsetArray64>
                    <type>var * int64</type>
                    <offsets><Index64 i="[0 2 3 4 6 ... 143 145 150 153 156]" offset="0" at="0x564882a4dfd0"/></offsets>
                    <content><NumpyArray format="l" shape="156" data="4 1 1 3 4 ... 3

Both `fromiter` and `fromjson` are implemented using `FillableArray`, the latter using the RapidJSON C++ library for deserialization.

In [50]:
# !wget https://scikit-hep.org/uproot/examples/HZZ.json

In [51]:
hzz = ak.fromjson("HZZ.json")
hzz

<Array [{jets: [], ... weight: 0.00876}] type='2421 * {"jets": var * {"px": floa...'>

In [52]:
for key in hzz.layout.keys():
    print("{:18s} {}".format(key, hzz[key].type))

jets               2421 * var * {"px": float64, "py": float64, "pz": float64, "E": float64, "id": bool}
muons              2421 * var * {"px": float64, "py": float64, "pz": float64, "E": float64, "q": int64, "iso": float64}
electrons          2421 * var * {"px": float64, "py": float64, "pz": float64, "E": float64, "q": int64, "iso": float64}
photons            2421 * var * {"px": float64, "py": float64, "pz": float64, "E": float64, "iso": float64}
MET                2421 * {"x": float64, "y": float64}
MC_hadronic_b      2421 * {"px": float64, "py": float64, "pz": float64}
MC_leptonic_b      2421 * {"px": float64, "py": float64, "pz": float64}
MC_hadronicW_q     2421 * {"px": float64, "py": float64, "pz": float64}
MC_hadronicW_qbar  2421 * {"px": float64, "py": float64, "pz": float64}
MC_lepton          2421 * {"px": float64, "py": float64, "pz": float64, "pdgid": int64}
MC_neutrino        2421 * {"px": float64, "py": float64, "pz": float64}
num_PV             2421 * int64
trigger_isomu

The loop over Python objects or JSON nodes was moved from Python into C++, so it's faster. However, the implementation requires vtable-lookups (the type is discovered at runtime), so it's not _a lot_ faster. There's room for specialized methods when the type is known in advance. (See [src/libawkward/io/root.cpp](https://github.com/scikit-hep/awkward-1.0/blob/master/src/libawkward/io/root.cpp) for a ${\tt vector}^N{\tt<number>}$ implementation.)

In general, turning rowwise data into columnar data is about 10× to 20× faster than it used to be.

In [53]:
import awkward as oldawkward
import json
asjson = open("HZZ.json").read()
aslist = json.load(open("HZZ.json"))

In [54]:
%%timeit

oldawkward.fromiter(aslist)

409 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%%timeit

ak.fromiter(aslist)

40.8 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%%timeit

oldawkward.fromiter(json.loads(asjson))

522 ms ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit

ak.fromjson(asjson)

28.6 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Awkward arrays in Numba

One of the motivating goals of the Awkward re-write was to incorporate Numba on the same footing.

In [58]:
import numba

@numba.jit(nopython=True)
def muon_sumpt(events):
    out = np.zeros(len(events), np.float64)
    i = 0
    for event in events:
        for muon in event["muons"]:
            out[i] += np.sqrt(muon["px"]**2 + muon["py"]**2)
        i += 1
    return out

In [59]:
muon_sumpt(hzz.layout)

array([91.91225969, 24.41791248, 83.40026411, ..., 33.46153652,
       63.61981771, 42.93994828])

Notice that we can write for loops on event _records_ and muon _records_—we don't have to take apart `JaggedArrays` and write algorithms on offsets and indexes.

My first version of the above raised segfaults because the `i += 1` was in the inner loop, rather than the outer loop (indentation error). Since it's Numba, I could debug it by running the pure Python version.

In [60]:
muon_sumpt.py_func

<function __main__.muon_sumpt(events)>

In [61]:
muon_sumpt.py_func(hzz.layout)

array([91.91225969, 24.41791248, 83.40026411, ..., 33.46153652,
       63.61981771, 42.93994828])

In [62]:
%%timeit

muon_sumpt(hzz.layout)

9.78 ms ± 864 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%timeit

muon_sumpt.py_func(hzz.layout)

210 ms ± 9.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So, a 20× speedup over Python without changing any code. Debug in Python, accelerate with Numba.

Awkward arrays could be a benefit to Numba users in general: Numba can handle complex data types by converting Python objects to and from equivalent structs, but that puts a translation burden at the entry and exit of every Numba function. Awkward leaves the data in the same form (big array buffers), transforming only its handles to the data. JSON → Awkward arrays → Numba could become an important workflow.

## FillableArray in Numba

The example above took an Awkward array into a Numba function and did some processing on it. To write data out, we can use `FillableArrays`.

In [64]:
@numba.jit(nopython=True)
def make_data(builder):
    for i in range(10):
        builder.beginrecord()

        builder.field("x")
        builder.real(i*1.1)

        builder.field("y")
        builder.beginlist()
        for j in range(i):
            builder.integer(j)
        builder.endlist()

        builder.endrecord()

    return builder

In [65]:
builder = ak.layout.FillableArray()
make_data(builder)

<FillableArray length="10" type="{"x": float64, "y": var * int64}"/>

In [66]:
builder.snapshot()

<RecordArray>
    <type>{"x": float64, "y": var * int64}</type>
    <field index="0" key="x">
        <NumpyArray format="d" shape="10" data="0 1.1 2.2 3.3 4.4 5.5 6.6 7.7 8.8 9.9" at="0x564892858680">
            <type>float64</type>
        </NumpyArray>
    </field>
    <field index="1" key="y">
        <ListOffsetArray64>
            <type>var * int64</type>
            <offsets><Index64 i="[0 0 1 3 6 ... 15 21 28 36 45]" offset="0" at="0x564892863e90"/></offsets>
            <content><NumpyArray format="l" shape="45" data="0 0 1 0 1 ... 4 5 6 7 8" at="0x564891bb1200">
                <type>int64</type>
            </NumpyArray></content>
        </ListOffsetArray64>
    </field>
</RecordArray>

In [67]:
ak.tolist(builder.snapshot())

[{'x': 0.0, 'y': []},
 {'x': 1.1, 'y': [0]},
 {'x': 2.2, 'y': [0, 1]},
 {'x': 3.3000000000000003, 'y': [0, 1, 2]},
 {'x': 4.4, 'y': [0, 1, 2, 3]},
 {'x': 5.5, 'y': [0, 1, 2, 3, 4]},
 {'x': 6.6000000000000005, 'y': [0, 1, 2, 3, 4, 5]},
 {'x': 7.700000000000001, 'y': [0, 1, 2, 3, 4, 5, 6]},
 {'x': 8.8, 'y': [0, 1, 2, 3, 4, 5, 6, 7]},
 {'x': 9.9, 'y': [0, 1, 2, 3, 4, 5, 6, 7, 8]}]

Since you can walk over data structures and create data structures (and later, assign fields to datasets like the old `Table`), you have complete freedom to manipulate data

   * at compiled-code speeds,
   * without having to leave the Python environment,
   * without having to rethink your algorithm in terms of array-at-a-time functions.

(Of course, you'll have the array-at-a-time functions, too.)

## Awkward arrays in C++

Since everything has been implemented in C++, Awkward 1.0 can be used in C++ programs. More importantly, we will (someday) be able to create Awkward arrays in C++ and access them in Python or vice-versa.

In [68]:
open("test-program.cpp", "w").write("""

#include <iostream>

#include "awkward/fillable/FillableArray.h"
#include "awkward/fillable/FillableOptions.h"

namespace ak = awkward;

int main(int, char**) {
  ak::FillableArray builder(ak::FillableOptions(1024, 2.0));
  for (int i = 0;  i < 10;  i++) {
    builder.beginrecord();

    builder.field_fast("x");    // (field_fast means don't check the whole string, just its pointer)
    builder.real(i*1.1);

    builder.field_fast("y");
    builder.beginlist();
    for (int j = 0;  j < i;  j++) {
      builder.integer(j);
    }
    builder.endlist();

    builder.endrecord();
  }
  
  std::cout << builder.snapshot()->tojson(false, 1) << std::endl;
  return 0;
}
""")

672

In [69]:
import pygments.formatters
import pygments.lexers.c_cpp
print(pygments.highlight(open("test-program.cpp").read(),
                         pygments.lexers.c_cpp.CppLexer(),
                         pygments.formatters.Terminal256Formatter()))

#include <iostream>

#include "awkward/fillable/FillableArray.h"
#include "awkward/fillable/FillableOptions.h"

namespace ak = awkward;

int main(int, char**) {
  ak::FillableArray builder(ak::FillableOptions(1024, 2.0));
  for (int i = 0;  i < 10;  i++) {
    builder.beginrecord();

    builder.field_fast("x");    // (field_fast means don't check the whole string, just its pointer)
    builder.real(i*1.1);

    builder.field_fast("y");
    builder.beginlist();
    for (int j = 0;  j < i;  j++) {
      builder.integer(j);
    }
    builder.endlist();

    builder.endrecord();
  }
  
  std::cout << builder.snapshot()->tojson(false, 1) << std::endl;
  return 0;
}



In [70]:
!g++ -I../../include -L../../awkward1 test-program.cpp -lawkward-static -lawkward-cpu-kernels-static -o test-program

In [71]:
!./test-program

[{"x":0.0,"y":[]},{"x":1.1,"y":[0]},{"x":2.2,"y":[0,1]},{"x":3.3,"y":[0,1,2]},{"x":4.4,"y":[0,1,2,3]},{"x":5.5,"y":[0,1,2,3,4]},{"x":6.6,"y":[0,1,2,3,4,5]},{"x":7.7,"y":[0,1,2,3,4,5,6]},{"x":8.8,"y":[0,1,2,3,4,5,6,7]},{"x":9.9,"y":[0,1,2,3,4,5,6,7,8]}]


## Identities: database-like index for arrays

In the [PartiQL toy language](https://github.com/jpivarski/PartiQL#readme), it became apparent that set operations, in which unique records are identified by reference, rather than by value, are important. They provide such operations as joins and lossless unions.

No set operations have been implemented, but implementing them will require an index that tracks particle identities through all other operations. This concept of an index is the primary distinction between an array library like NumPy and a relational library like Pandas. In Awkward, this index is called an `Identity` and can optionally be attached to arrays.

**Note:** this interface is the most likely to change. Identities have only been implemented at this early stage so that they don't have to be painfully retrofitted later.

In [80]:
hzzlayout = ak.fromjson("HZZ.json").layout

In [81]:
hzzlayout.setid()
hzzlayout.id

<Identity32 ref="5" fieldloc="[]" width="1" offset="0" length="2421" at="0x564890270540"/>

In [88]:
hzzlayout.field("muons").content.field("px").id

<Identity64 ref="7" fieldloc="[(0, 'muons') (1, 'px')]" width="2" offset="0" length="3825" at="0x5648900cc5f0"/>

In [89]:
np.asarray(hzzlayout.field("muons").content.field("px").id)

array([[   0,    0],
       [   0,    1],
       [   1,    0],
       ...,
       [2418,    0],
       [2419,    0],
       [2420,    0]], dtype=int64)

An `Identity` is a 2-dimensional array with the same structure as a Pandas row `MultiIndex` with a `fieldloc` for the nested columns. They're equivalent to paths from root (wherever you called `setid`) to the element in question.

In [91]:
hzzlayout[1000, "muons", 1].location

(1000, 'muons', 1)

As a nice side-effect of having indexes, we can give better error messages about where an indexing error occurs. You might use `Identities` just for debugging.

In [84]:
# Indexing error with an Identity:
try:
    hzzlayout[1000, "muons", 2]
except Exception as err:
    print(err)

in ListArray64 at id[1000, "muons"] attempting to get 2, index out of range


In [92]:
# Indexing error without an Identity:
try:
    ak.fromjson("HZZ.json").layout[1000, "muons", 2]
except Exception as err:
    print(err)

in ListArray64 attempting to get 2, index out of range


When the array goes through any kind of transformation, such as the boolean filter below, the `Identity` is similarly selected.

In [113]:
mask = np.random.randint(0, 100, len(hzzlayout)) == 0
mask

array([False, False, False, ..., False, False, False])

In [114]:
selected = hzzlayout[mask]
np.asarray(selected.id)

array([[  17],
       [ 192],
       [ 366],
       [ 492],
       [ 494],
       [ 711],
       [ 743],
       [ 868],
       [1094],
       [1121],
       [1281],
       [1354],
       [1368],
       [1399],
       [1614],
       [1630],
       [1666],
       [1718],
       [1824],
       [2218]], dtype=int32)

In this way, the `Identity` acts as a set of labels that are permanently glued to the array elements.